## Imports

In [1]:
# python imports
import os
import re
import json
import random

# third-party imports
import numpy as np
import pandas as pd

## Config

In [2]:
NETWORK_NAME = 'Network02'
PREFIX = 'scenario-2'

DRAW_TITLE = False
SAVE_FIG = False

OUTPUT_FIG = 'imgs'
EXTENSION = 'pdf'

## Global Variables

In [3]:
VUL_COLUMNS = ['cve_id', 'base_score', 'base_severity', 'vendor', 'cve_published_date']
CTI_COLUMNS = ['epss', 'exploit_count', 'exploit_published_date', 'attack_type', 'security_advisory', 'mitre_top_25', 'owasp_top_10']
CTX_COLUMNS = ['topology', 'asset_type', 'environment', 'sensitive_data', 'end_of_life', 'critical_asset'] 

SEVERITY_LABELS = ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']
RISK_LABELS = ['LOW', 'MODERATE', 'IMPORTANT', 'CRITICAL']

COLORS = {
    'LOW': '#14943D',
    'MEDIUM': '#FDBE0E',
    'MODERATE': '#FDBE0E',
    'HIGH': '#FC7303',
    'IMPORTANT': '#FC7303',
    'CRITICAL': '#F20101'
}

## Utils

In [4]:
def print_value_counts(df, cols, normalize=False):
    for col in cols:
        val_c = df[col].value_counts(normalize=normalize)

        print(f'\n# {col.upper()}')
        for index, value in zip(val_c.index, val_c.values):
            print(f' - {index}: {value:.2f}%')

In [5]:
def track_candidate(iround, cve_id, cols):
    files = os.listdir(f'../output/{NETWORK_NAME}/')

    cvss_history = None
    frape_history = None
    
    with open(f'../output/{NETWORK_NAME}/{files[iround]}') as f:

        file = json.load(f)

        cvss_history = file['history']['cvss']
        frape_history = file['history']['frape']
        
    print(f'\nloaded file {NETWORK_NAME}/{files[iround]} of independent round {iround}º\n')
    
    for index, (cvss, frape) in enumerate(zip(cvss_history, frape_history)):
        
        cvss = pd.DataFrame.from_records(cvss)
        frape = pd.DataFrame.from_records(frape)
        
        if index == 0:
            candidate = frape.loc[frape['cve_id'] == cve_id][cols].squeeze(axis=0)
            print(f'{candidate}\n')
        
        if cve_id in cvss.cve_id.values:
            print(f'- fixed in iteration {index}º for cvss')
        
        if cve_id in frape.cve_id.values:
            print(f'- fixed in iteration {index}º for frape')

## Analysis

In [6]:
track_candidate(20, 'CVE-2020-3956', VUL_COLUMNS + CTI_COLUMNS)


loaded file Network02/environment-2023-07-08--15-21-03.json of independent round 20º

cve_id                                CVE-2020-3956
base_score                                      8.8
base_severity                                  HIGH
vendor                                        linux
cve_published_date                       2020-05-20
epss                                        0.52583
exploit_count                                   1.0
exploit_published_date                   2020-06-02
attack_type               ['remote code execution']
security_advisory                                 0
mitre_top_25                                      0
owasp_top_10                                      1
Name: 0, dtype: object

- fixed in iteration 0º for frape
- fixed in iteration 28º for cvss


In [7]:
track_candidate(15, 'CVE-2021-30613', VUL_COLUMNS + CTX_COLUMNS)


loaded file Network02/environment-2023-07-08--16-13-40.json of independent round 15º

cve_id                CVE-2021-30613
base_score                       8.8
base_severity                   HIGH
vendor                     microsoft
cve_published_date        2021-09-03
topology                         DMZ
asset_type                    SERVER
environment               PRODUCTION
sensitive_data                     1
end_of_life                        1
critical_asset                     0
Name: 26, dtype: object

- fixed in iteration 0º for frape
- fixed in iteration 23º for cvss
